In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import project_env as pe
import sklearn as sklearn
%matplotlib inline

In [6]:
data_train = pd.read_csv('data_train_enc.csv', encoding = "ISO-8859-1", low_memory=False)
data_val = pd.read_csv('data_val_enc.csv', encoding = "ISO-8859-1", low_memory=False)

# Target Variable

In [15]:
#target variable of rearrest in 2 years
X_train, y_train = pe.create_target(data_train, years = 2)
X_val, y_val = pe.create_target(data_val, years = 2)

In [8]:
unique, counts = np.unique(y_train, return_counts = True)
percent = counts[1]/counts.sum()
print('Training: %s percent were rearrested.' %(percent*100))

unique, counts = np.unique(y_val, return_counts = True)
percent = counts[1]/counts.sum()
print('Test: %s percent were rearrested.' %(percent*100))


Training: 55.6245941507 percent were rearrested.
Test: 55.273745862 percent were rearrested.


In [16]:
#remove non-feature columns
X_train=X_train.drop(['BOFI_NBR','SCREENING_DISP_CODE','UNIQUE_ID','NEXT_ARREST_TIME'], axis=1)
X_val=X_val.drop(['BOFI_NBR','SCREENING_DISP_CODE','UNIQUE_ID','NEXT_ARREST_TIME'], axis=1)

#Using year and month as predictive variables
X_train=X_train.drop(['DOB','ARREST_DATE','POLICE_RPT_DATE','SCREENING_DISP_DATE',\
                      'BAR_ADMISSION','ARREST_DATE_y'], axis=1)
X_val=X_val.drop(['DOB','ARREST_DATE','POLICE_RPT_DATE','SCREENING_DISP_DATE',\
                  'BAR_ADMISSION','ARREST_DATE_y'], axis=1)


In [17]:
#check for missing val
X_train.count().min()==X_train.count().max()

True

# Gradient Boosted Trees

In [18]:
from sklearn.ensemble import GradientBoostingClassifier
gbt = GradientBoostingClassifier()
gbt = gbt.fit(X_train,y_train)

pred_train = gbt.predict(X_train)
pred_val = gbt.predict(X_val)


In [19]:
#accuracy

accu_train = sklearn.metrics.accuracy_score(y_train,pred_train)
accu_test = sklearn.metrics.accuracy_score(y_val,pred_val)

print("Accuracy on Training Dataset: {}".format(accu_train))
print("Accuracy on Val Dataset: {}".format(accu_test))

Accuracy on Training Dataset: 0.7435541578069494
Accuracy on Val Dataset: 0.7387318563789153


In [20]:
precision = sklearn.metrics.precision_score(y_val,pred_val)
recall = sklearn.metrics.recall_score(y_val,pred_val)
f_score = sklearn.metrics.f1_score(y_val,pred_val)

print("Precision: {}".format(precision))
print("Recall: {}".format(recall))
print("F-Score: {}".format(f_score))

Precision: 0.7421101616041966
Recall: 0.8081636413894776
F-Score: 0.7737297106563161


In [21]:
#confusion matrix for val set
cm = sklearn.metrics.confusion_matrix(y_val,pred_val)
cm

array([[5734, 3048],
       [2082, 8771]])